# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
file = "../output_data/cities"
cities_df = pd.read_csv(file)

cities_df.describe()

,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,5.490000e+02
mean,274.000000,20.105550,15.598938,20.552860,69.255009,51.440801,3.373515,1.627593e+09
std,158.626921,33.127902,91.220836,8.231477,21.159729,38.639437,2.404553,2.310453e+02
min,0.000000,-54.800000,-179.166700,-3.120000,11.000000,0.000000,0.000000,1.627592e+09
25%,137.000000,-7.938600,-64.798100,14.390000,55.000000,10.000000,1.660000,1.627592e+09
50%,274.000000,23.116700,18.423200,21.410000,75.000000,55.000000,2.710000,1.627593e+09
75%,411.000000,48.005600,96.733300,26.280000,86.000000,90.000000,4.520000,1.627593e+09
max,548.000000,78.218600,179.316700,41.910000,100.000000,100.000000,16.710000,1.627593e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
gmaps.configure(api_key=g_key)

In [43]:
#generate list of humidity
list_humidity = cities_df["Humidity"].tolist()

#establish google map
fig = gmaps.figure()

#provide lat/long
locations = cities_df[["Lat", "Lng"]]

#state heatmap required, weight based on humidity
heat_layer = gmaps.heatmap_layer(locations, weights=list_humidity)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [45]:
low_wind_df = len(cities_df[cities_df["Wind Speed"] <= 10])
good_temp = len(cities_df[(cities_df["Max Temp"] > 15) & (cities_df["Max Temp"] < 30)])
zero_clouds = len(cities_df[cities_df["Cloudiness"] == 0])

ideal_criteria = cities_df[(cities_df["Wind Speed"] <= 10) &
                           (cities_df["Max Temp"] > 15) &
                           (cities_df["Max Temp"] < 30) &
                           (cities_df["Cloudiness"] == 0)]

f"{low_wind_df} cities have ideal wind speeds, {good_temp} have ideal temperatures and {zero_clouds} have ideal cloudiness!"

ideal_criteria.head(20).reset_index(drop=True)

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,14,Grindavik,63.8424,-22.4338,17.26,77,0,2.57,IS,1627592253
1,27,Vila,42.0304,-8.1588,18.75,79,0,0.94,PT,1627592255
2,29,Asău,46.4333,26.4000,21.53,76,0,1.48,RO,1627592255
3,62,Santiago,-33.4569,-70.6483,23.69,11,0,2.06,CL,1627592106
4,67,Koungou,-12.7336,45.2042,24.84,69,0,5.14,YT,1627592319
5,68,East London,-33.0153,27.9116,15.48,71,0,5.64,ZA,1627592320
6,73,Kiryat Gat,31.6100,34.7642,27.44,75,0,1.79,IL,1627592320
7,77,Arrondissement de Saint-Denis,48.9167,2.3333,20.08,61,0,0.51,FR,1627592275
8,78,Manjacaze,-24.7117,33.8828,15.16,77,0,2.70,MZ,1627592321
9,89,Kasongo-Lunda,-6.4667,16.8167,22.53,25,0,0.34,CD,1627592323


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
